In [ ]:
# Load necessary libraries
import numpy as np
import pandas as pd 
import random as rn

# Tensorflow
import tensorflow.random as tfr
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D, MaxPooling2D, BatchNormalization
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Chart
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import seaborn as sns
import glob

from skimage import color, exposure
from sklearn.metrics import classification_report

from PIL import Image
import os
import cv2

# Setting the same seed for repeatability

seed = 123

np.random.seed(seed) 
rn.seed(seed)
tfr.set_seed(seed)

# Display graphs in a Jupyter
%matplotlib inline

print("Imported")

In [ ]:
# Create a path for the inputs
data_path = '../input/dogs-cats-images/dataset/'
data_path

train_path = data_path + 'training_set/'
test_path = data_path + 'test_set/'

In [ ]:
# View a couple images
import matplotlib.image as mpimg 

img_cat = mpimg.imread('../input/dogs-cats-images/dataset/training_set/cats/cat.1.jpg') 
img_dog = mpimg.imread('../input/dogs-cats-images/dataset/training_set/dogs/dog.1004.jpg') 

fig = plt.figure()
ax1 = fig.add_subplot(2,2,1)
ax1.imshow(img_cat)
ax2 = fig.add_subplot(2,2,2)
ax2.imshow(img_dog)

In [ ]:
# Check out some info on the images
print(img_cat.size)
print(img_cat.dtype)
print(img_cat.shape)

In [ ]:
# Set image size
img_size = 64

In [ ]:
from skimage.color import rgb2gray

In [ ]:
# Function to read in the data, resize, and add grayscale
def read_data(data_paths):
    for data_path in data_paths:
        labels = ['cats', 'dogs']
        images = []
        y = []
        for label in labels:
            curr_path = data_path + label
            for img in os.listdir(curr_path):
                if ('DS' not in img):
                    image_path = os.path.join(curr_path, img)
                    grayscale =  cv2.resize(cv2.imread(image_path), (img_size, img_size))
                    image = rgb2gray(grayscale)
                    if image is not None:
                        images.append([image, label])
                
    images = np.asarray(images)
    return images


In [ ]:
"""
# Another function to read in the data, resize, and use numpy mean to change to black and white (sort of) 
def read_data(data_paths):
    for data_path in data_paths:
        labels = ['cats', 'dogs']
        images = []
        y = []
        for label in labels:
            curr_path = data_path + label
            for img in os.listdir(curr_path):
                if ('DS' not in img):
                    image_path = os.path.join(curr_path, img)
                    color_img =  cv2.resize(cv2.imread(image_path), (img_size, img_size))
                    image = np.mean(color_img, axis=2)
                    if image is not None:
                        images.append([image, label])
                
    images = np.asarray(images)
    return images
"""

In [ ]:
"""
# Original function to read in the data 
def read_data(data_paths):
    for data_path in data_paths:
        labels = ['cats', 'dogs']
        images = []
        y = []
        for label in labels:
            curr_path = data_path + label
            for img in os.listdir(curr_path):
                if ('DS' not in img):
                    image_path = os.path.join(curr_path, img)
                    image =  cv2.resize(cv2.imread(image_path), (img_size, img_size))
                    if image is not None:
                        images.append([image, label])
                
    images = np.asarray(images)
    return images
"""

In [ ]:
# Run my read_data function
train = read_data([train_path])
test = read_data([test_path])

In [ ]:
# Shuffle the data
for i in range(10):
    np.random.shuffle(train)
    np.random.shuffle(test)

In [ ]:
# Create data frames 
train_df = pd.DataFrame(train, columns=['image', 'label'])
test_df = pd.DataFrame(test, columns = ['image', 'label'])

In [ ]:
# Make sure it worked
train_df.head()

In [ ]:
for i in range(5):
    plt.imshow(train_df.image[i])
    plt.show()
#    cmap=plt.get_cmap('gray')

In [ ]:
# Look at size and balance
train_df.groupby("label").count()

In [ ]:
# Make sure all is well here
test_df.head()

In [ ]:
# Image from dataframe
plt.title('Dog or a cat?', size=18)
plt.imshow(test_df.image[3])
plt.show()

In [ ]:
# Looks even again
test_df.groupby("label").count()

In [ ]:
# Function to change dogs and cats to 0 and 1.
def animal_type(label):
    if label == 'dogs':
        return 0
    else:
        return 1

In [ ]:
# Function to create X and y split
def splitdata(data):
    X = []
    y = []
    for i, (val, label) in enumerate(data):
        X.append(val)
        y.append(animal_type(label))
    return np.array(X), np.array(y)

In [ ]:
# Shuffle again and run function
X_train, y_train = splitdata(train)
X_test, y_test = splitdata(test)

In [ ]:
X_train.shape

In [ ]:
X_train[0].shape

In [ ]:
# All 1's and 0's
y_train

In [ ]:
y_train.shape

In [ ]:
y_train[0].shape

In [ ]:
# Reshaping, Greyscale, and Normalizing function
#def preprocesing_to_mlp(data):
#    data1 = color.rgb2gray(data).reshape(-1, img_size * img_size).astype('float32')
#    data1 /= 255
#    return data1

In [ ]:
# Run preprocesing function
#X_train = preprocesing_to_mlp(X_train)
#X_test = preprocesing_to_mlp(X_test)

In [ ]:
print(len(X_train))
print(len(X_test))

In [ ]:
# Image from array
plt.title('Dog or a cat?', size=18)
plt.imshow(X_train[0])
plt.show()

In [ ]:
# Only these picture are black and white. Above is what they really look like.
for i in range(5):
    plt.imshow(X_train[i], cmap=plt.get_cmap('gray'))
    plt.show()

In [ ]:
plt.figure(figsize = (20,6))
for i in range(5):
    plt.subplot(1,5,i+1)
    plt.imshow(X_train[i])
plt.show()

### From Jeff

In [ ]:
# Model
model_1 = Sequential()
model_1.add(Flatten(input_shape=(64, 64)))
model_1.add(Dense(256, activation='relu'))
model_1.add(Dense(256, activation='relu'))
model_1.add(Dense(128, activation='relu'))
model_1.add(Dense(10, activation='softmax'))

In [ ]:
# Summary
model_1.summary()

In [ ]:
# Compile
model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Fit the model
training_results = model_1.fit(X_train, y_train,
                               epochs=100, batch_size=128,
                               validation_data=(X_test, y_test))

## My own code

In [ ]:
# Build the basic  NN model.
model = Sequential([
  Dense(64, activation='relu', input_shape=(64,64)),
  Dense(32, activation='relu'),
  Dense(32, activation='softmax'),
])

In [ ]:
# CNN
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.3))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.2))
model.add(Conv2D(filters = 126, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.15))
model.add(Flatten())
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dropout(rate = 0.15))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(rate = 0.1))
model.add(Dense(units = 64, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
# Fit the model
model.fit(X_train, 
          y_train, 
          epochs=5, 
          batch_size=40)

In [ ]:
# directory with our pictures
train_dir = "../input/dogs-cats-images/dataset/test_set"
test_dir = "../input/dogs-cats-images/dataset/training_set"
# directory with our training cat pictures
train_cats_dir =  train_dir + '/cats' 
train_dogs_dir = train_dir + '/dogs'
# directory with our validation cat pictures
test_cats_dir = test_dir + '/cats'  
test_dogs_dir = test_dir + '/dogs' 

In [ ]:
data_generator = ImageDataGenerator(rescale = 1.0/255.0, zoom_range = 0.2)

In [ ]:
batch_size = 32
training_data = data_generator.flow_from_directory(directory = train_dir,
                                                   target_size = (64, 64),
                                                   batch_size = batch_size,
                                                   class_mode = 'binary')
testing_data = data_generator.flow_from_directory(directory = test_dir,
                                                  target_size = (64, 64),
                                                  batch_size = batch_size,
                                                  class_mode = 'binary')

In [ ]:
from keras.layers import Conv2D, MaxPooling2D


In [ ]:
model_cnn = Sequential()
model_cnn.add(Conv2D(32, (3, 3), activation = 'relu', input_shape = (28, 28, 1)))
model_cnn.add(MaxPooling2D((2, 2)))
model_cnn.add(Conv2D(64, (3, 3), activation = 'relu'))
model_cnn.add(MaxPooling2D((2, 2)))
model_cnn.add(Conv2D(64, (3, 3), activation = 'relu'))
model_cnn.add(Flatten())
model_cnn.add(Dense(128, activation = 'relu'))
model_cnn.add(Dense(64, activation = 'relu'))
model_cnn.add(Dense(10, activation = 'softmax'))

In [ ]:
model_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
training_results_cnn = model_cnn.fit(X_train.reshape(60000, 28, 28, -1), y_binary_train,
                               epochs=10, batch_size=128,
                               validation_data=(X_test.reshape(10000, 28, 28, -1), y_binary_test))

In [ ]:
import os
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')])

In [ ]:
 #importing our deep learing libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Building our CNN
model = Sequential() 
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = training_data.image_shape))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.3))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.2))
model.add(Conv2D(filters = 126, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.15))
model.add(Flatten())
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dropout(rate = 0.15))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(rate = 0.1))
model.add(Dense(units = len(set(training_data.classes)), activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [1]:
# I also used this notebook from the pneumonia dataset to help.
# https://www.kaggle.com/sanwal092/intro-to-cnn-using-keras-to-predict-pneumonia
    